In [2]:
#! Does not work on local - problems with the env
from time import time
import pandas as pd

from omegaconf import OmegaConf
import torch
from models.clip.extract_clip import ExtractCLIP
from utils.utils import build_cfg_path

ModuleNotFoundError: No module named 'omegaconf'

In [1]:
# For windows
import os
import pandas as pd
import time
import torch
import pickle
import sys


In [8]:
# utils_path = os.path.abspath(os.path.join('..'))
# if utils_path not in sys.path:
sys.path.append(sys.path[0] + '/..')

In [12]:
from src.utils import get_sec, read_pickle
from src.constants import DATA_ROOT

In [2]:
DATA_ROOT = '../2g1n6qdydwa9u22shpxqzp0t8m/'
participants = os.listdir(DATA_ROOT)
participants.remove('other')
participants.remove('temp')
video_paths = []
for p in sorted(participants):
    video_path = DATA_ROOT + p + '/videos/'
    videos = sorted(os.listdir(video_path))
    videos = [video_path + v for v in videos]
    video_paths.extend(videos)

print(len(video_paths))
print(video_paths[:10])


268
['../2g1n6qdydwa9u22shpxqzp0t8m/P01/videos/P01_101.MP4', '../2g1n6qdydwa9u22shpxqzp0t8m/P01/videos/P01_102.MP4', '../2g1n6qdydwa9u22shpxqzp0t8m/P01/videos/P01_103.MP4', '../2g1n6qdydwa9u22shpxqzp0t8m/P01/videos/P01_104.MP4', '../2g1n6qdydwa9u22shpxqzp0t8m/P01/videos/P01_105.MP4', '../2g1n6qdydwa9u22shpxqzp0t8m/P01/videos/P01_106.MP4', '../2g1n6qdydwa9u22shpxqzp0t8m/P01/videos/P01_107.MP4', '../2g1n6qdydwa9u22shpxqzp0t8m/P01/videos/P01_108.MP4', '../2g1n6qdydwa9u22shpxqzp0t8m/P01/videos/P01_109.MP4', '../2g1n6qdydwa9u22shpxqzp0t8m/P02/videos/P02_101.MP4']


## Pilot study

In [17]:
rgb = read_pickle('data/i3d/P01_101_rgb.pkl')
fps = read_pickle('data/i3d/P01_101_fps.pkl')
ts = read_pickle('data/i3d/P01_101_timestamps_ms.pkl')

In [18]:
print(type(rgb))
print(type(fps))
print(type(ts))


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [19]:
print(ts.shape, fps.shape, rgb.shape)

(1430,) () (1430, 1024)


In [20]:
print(fps)

50.0


In [22]:
print(ts)

[1.28000e+03 2.56000e+03 3.84000e+03 ... 1.82784e+06 1.82912e+06
 1.83040e+06]


In [21]:
for feat, time in zip(rgb, ts):
    print(f'feature.shape = {feat.shape}, timestamp = {time}')

feature.shape = (1024,), timestamp = 1280.0
feature.shape = (1024,), timestamp = 2560.0
feature.shape = (1024,), timestamp = 3840.0000000000005
feature.shape = (1024,), timestamp = 5120.0
feature.shape = (1024,), timestamp = 6400.0
feature.shape = (1024,), timestamp = 7680.000000000001
feature.shape = (1024,), timestamp = 8960.0
feature.shape = (1024,), timestamp = 10240.0
feature.shape = (1024,), timestamp = 11520.000000000002
feature.shape = (1024,), timestamp = 12800.0
feature.shape = (1024,), timestamp = 14080.0
feature.shape = (1024,), timestamp = 15360.000000000002
feature.shape = (1024,), timestamp = 16640.0
feature.shape = (1024,), timestamp = 17920.0
feature.shape = (1024,), timestamp = 19200.0
feature.shape = (1024,), timestamp = 20480.0
feature.shape = (1024,), timestamp = 21760.0
feature.shape = (1024,), timestamp = 23040.000000000004
feature.shape = (1024,), timestamp = 24320.0
feature.shape = (1024,), timestamp = 25600.0
feature.shape = (1024,), timestamp = 26880.00000000

In [3]:
with open('../../data/train100_mod.pkl', 'rb') as handle:
    df = pickle.load(handle)

df.head()

,video_id,root_dir,narration,start_frame,end_frame,verb_class,noun_class
0,P01_102,../2g1n6qdydwa9u22shpxqzp0t8m/P01,take knife and plate,56,381,0,4
1,P01_102,../2g1n6qdydwa9u22shpxqzp0t8m/P01,move spatula and colander,381,1493,11,20
2,P01_102,../2g1n6qdydwa9u22shpxqzp0t8m/P01,take knife,1493,11231,0,4
3,P01_102,../2g1n6qdydwa9u22shpxqzp0t8m/P01,wipe surface,11231,11281,2,42
4,P01_102,../2g1n6qdydwa9u22shpxqzp0t8m/P01,move glass,11281,11343,11,10


In [4]:
df.tail()

,video_id,root_dir,narration,start_frame,end_frame,verb_class,noun_class
32220,P35_109,../2g1n6qdydwa9u22shpxqzp0t8m/P35,put down scissors,14769,15007,1,79
32221,P35_109,../2g1n6qdydwa9u22shpxqzp0t8m/P35,pick up mug,15007,15081,0,13
32222,P35_109,../2g1n6qdydwa9u22shpxqzp0t8m/P35,put down mug,15081,15119,1,13
32223,P35_109,../2g1n6qdydwa9u22shpxqzp0t8m/P35,pick up mug,15119,15219,0,13
32224,P35_109,../2g1n6qdydwa9u22shpxqzp0t8m/P35,put down mug,15219,56475,1,13


For debugging

In [55]:
with open('../../data/train100_version_fix.pkl', 'rb') as handle:
    df = pickle.load(handle)

df.head()

,participant_id,video_id,narration_timestamp,start_timestamp,stop_timestamp,start_frame,stop_frame,narration,verb,verb_class,noun,noun_class,all_nouns,all_noun_classes
narration_id,,,,,,,,,,,,,,
P01_102_0,P01,P01_102,00:00:01.100,00:00:00.54,00:00:02.23,27,111,take knife and plate,take,0,knife,4,"[knife, plate]","[4, 2]"
P01_102_1,P01,P01_102,00:00:07.600,00:00:07.49,00:00:09.05,374,452,move spatula and colander,move,11,spatula,20,"[spatula, colander]","[20, 39]"
P01_102_10,P01,P01_102,00:00:29.851,00:00:29.66,00:00:30.84,1483,1542,take knife,take,0,knife,4,[knife],[4]
P01_102_100,P01,P01_102,00:03:44.600,00:03:44.45,00:03:45.00,11222,11250,wipe surface,wipe,2,surface,42,[surface],[42]
P01_102_101,P01,P01_102,00:03:45.600,00:03:45.61,00:03:46.01,11280,11300,move glass,move,11,glass,10,[glass],[10]


In [16]:
video_df = pd.read_csv('../../data/epic-kitchens-100-annotations/EPIC_100_video_info.csv')

In [17]:
video_df[video_df.video_id == 'P01_102']

,video_id,duration,fps,resolution
20,P01_102,274.92,50.0,1920x1080


In [56]:
df = df.reset_index().sort_values(by=['video_id', 'narration_timestamp'])

In [57]:
df_feats = df[['video_id', 'participant_id', 'narration_timestamp', 'narration', 'verb_class', 'noun_class']]

In [58]:
df_feats.head(10)

,video_id,participant_id,narration_timestamp,narration,verb_class,noun_class
0,P01_102,P01,00:00:01.100,take knife and plate,0,4
1,P01_102,P01,00:00:07.600,move spatula and colander,11,20
39,P01_102,P01,00:00:10.350,open bin,3,36
50,P01_102,P01,00:00:11.600,take knife,0,4
61,P01_102,P01,00:00:12.600,move orange skin into the bin,11,48
72,P01_102,P01,00:00:13.850,put down plate,1,2
83,P01_102,P01,00:00:15.600,close bin,4,36
94,P01_102,P01,00:00:16.600,take sponge,0,9
105,P01_102,P01,00:00:17.100,wash spatula,2,20
116,P01_102,P01,00:00:29.600,put down spatula,1,20


In [76]:
new_df = pd.merge(df_feats, video_df, how='left', on='video_id')

In [77]:
new_df.head(10)

,video_id,participant_id,narration_timestamp,narration,verb_class,noun_class,duration,fps,resolution
0,P01_102,P01,00:00:01.100,take knife and plate,0,4,274.92,50.0,1920x1080
1,P01_102,P01,00:00:07.600,move spatula and colander,11,20,274.92,50.0,1920x1080
2,P01_102,P01,00:00:10.350,open bin,3,36,274.92,50.0,1920x1080
3,P01_102,P01,00:00:11.600,take knife,0,4,274.92,50.0,1920x1080
4,P01_102,P01,00:00:12.600,move orange skin into the bin,11,48,274.92,50.0,1920x1080
5,P01_102,P01,00:00:13.850,put down plate,1,2,274.92,50.0,1920x1080
6,P01_102,P01,00:00:15.600,close bin,4,36,274.92,50.0,1920x1080
7,P01_102,P01,00:00:16.600,take sponge,0,9,274.92,50.0,1920x1080
8,P01_102,P01,00:00:17.100,wash spatula,2,20,274.92,50.0,1920x1080
9,P01_102,P01,00:00:29.600,put down spatula,1,20,274.92,50.0,1920x1080


In [78]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29540 entries, 0 to 29539
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   video_id             29540 non-null  object 
 1   participant_id       29540 non-null  object 
 2   narration_timestamp  29540 non-null  object 
 3   narration            29540 non-null  object 
 4   verb_class           29540 non-null  int64  
 5   noun_class           29540 non-null  int64  
 6   duration             29540 non-null  float64
 7   fps                  29540 non-null  float64
 8   resolution           29540 non-null  object 
dtypes: float64(2), int64(2), object(5)
memory usage: 2.3+ MB


In [79]:
from utils import get_sec

new_df['start_frame'] = new_df.apply(lambda row: 
                                     int(get_sec(row['narration_timestamp']) * row['fps']) + 1,
                                     axis=1)


In [80]:
new_df['end_frame'] = new_df.groupby('video_id')['start_frame'].shift(-1, fill_value=0)

In [81]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29540 entries, 0 to 29539
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   video_id             29540 non-null  object 
 1   participant_id       29540 non-null  object 
 2   narration_timestamp  29540 non-null  object 
 3   narration            29540 non-null  object 
 4   verb_class           29540 non-null  int64  
 5   noun_class           29540 non-null  int64  
 6   duration             29540 non-null  float64
 7   fps                  29540 non-null  float64
 8   resolution           29540 non-null  object 
 9   start_frame          29540 non-null  int64  
 10  end_frame            29540 non-null  int64  
dtypes: float64(2), int64(4), object(5)
memory usage: 2.7+ MB


In [82]:
new_df.head(15)

,video_id,participant_id,narration_timestamp,narration,verb_class,noun_class,duration,fps,resolution,start_frame,end_frame
0,P01_102,P01,00:00:01.100,take knife and plate,0,4,274.92,50.0,1920x1080,56,381
1,P01_102,P01,00:00:07.600,move spatula and colander,11,20,274.92,50.0,1920x1080,381,518
2,P01_102,P01,00:00:10.350,open bin,3,36,274.92,50.0,1920x1080,518,581
3,P01_102,P01,00:00:11.600,take knife,0,4,274.92,50.0,1920x1080,581,631
4,P01_102,P01,00:00:12.600,move orange skin into the bin,11,48,274.92,50.0,1920x1080,631,693
5,P01_102,P01,00:00:13.850,put down plate,1,2,274.92,50.0,1920x1080,693,781
6,P01_102,P01,00:00:15.600,close bin,4,36,274.92,50.0,1920x1080,781,831
7,P01_102,P01,00:00:16.600,take sponge,0,9,274.92,50.0,1920x1080,831,856
8,P01_102,P01,00:00:17.100,wash spatula,2,20,274.92,50.0,1920x1080,856,1481
9,P01_102,P01,00:00:29.600,put down spatula,1,20,274.92,50.0,1920x1080,1481,1493


In [83]:
new_df.iloc[127]

video_id                    P01_103
participant_id                  P01
narration_timestamp    00:00:02.103
narration                close door
verb_class                        4
noun_class                        3
duration                     178.02
fps                            50.0
resolution                1920x1080
start_frame                     106
end_frame                       381
Name: 127, dtype: object

In [84]:
new_df[new_df.end_frame == 0]

,video_id,participant_id,narration_timestamp,narration,verb_class,noun_class,duration,fps,resolution,start_frame,end_frame
126,P01_102,P01,00:04:32.100,close fridge,4,12,274.92,50.0,1920x1080,13606,0
224,P01_103,P01,00:02:55.601,put down cup,1,13,178.02,50.0,1920x1080,8781,0
317,P01_104,P01,00:04:03.601,put down cloth,1,17,245.64,50.0,1920x1080,12181,0
1107,P01_105,P01,00:35:18.110,put down chopsticks,1,80,2120.36,50.0,1920x1080,105906,0
1354,P01_106,P01,00:12:01.100,close drawer,4,8,723.38,50.0,1920x1080,36056,0
...,...,...,...,...,...,...,...,...,...,...,...
28898,P30_113,P30,00:06:04.852,put mug,1,13,366.64,50.0,1920x1080,18243,0
28937,P30_114,P30,00:01:55.100,empty washing up bowl,20,7,122.32,50.0,1920x1080,5756,0
29315,P37_101,P37,00:27:55.903,shake hands,15,11,1681.46,50.0,1920x1080,83796,0
29465,P37_102,P37,00:11:18.902,put knife down,1,4,682.62,50.0,1920x1080,33946,0


In [85]:
new_df['end_frame'] = new_df.apply(lambda row: int(row['duration'] * row['fps']) + 1 if row['end_frame'] == 0 else row['end_frame'],
                                   axis=1)

In [86]:
new_df.iloc[126]

video_id                    P01_102
participant_id                  P01
narration_timestamp    00:04:32.100
narration              close fridge
verb_class                        4
noun_class                       12
duration                     274.92
fps                            50.0
resolution                1920x1080
start_frame                   13606
end_frame                     13747
Name: 126, dtype: object

In [92]:
dataset = new_df[['video_id', 'participant_id', 'narration', 'start_frame', 'end_frame', 'verb_class', 'noun_class']]
dataset = dataset.to_numpy()
print(len(dataset))

29540


In [93]:
for i in range(10):
    print(dataset[i])

['P01_102' 'P01' 'take knife and plate' 56 381 0 4]
['P01_102' 'P01' 'move spatula and colander' 381 518 11 20]
['P01_102' 'P01' 'open bin' 518 581 3 36]
['P01_102' 'P01' 'take knife' 581 631 0 4]
['P01_102' 'P01' 'move orange skin into the bin' 631 693 11 48]
['P01_102' 'P01' 'put down plate' 693 781 1 2]
['P01_102' 'P01' 'close bin' 781 831 4 36]
['P01_102' 'P01' 'take sponge' 831 856 0 9]
['P01_102' 'P01' 'wash spatula' 856 1481 2 20]
['P01_102' 'P01' 'put down spatula' 1481 1493 1 20]


In [ ]:
video_id = 

End debugging

In [19]:
# Load training file
with open('../epic-kitchens-100-annotations/EPIC_100_train.pkl', 'rb') as f:
    df_train = pickle.load(f)

with open('../epic-kitchens-100-annotations/EPIC_100_validation.pkl', 'rb') as f:
    df_validation = pickle.load(f)

Some notes about the data below:
- `df_train` contains both old videos and videos from the extension
- `df_validation` contains only videos from EPIC-Kitchens-55
- test timestamps are only for videos from the extension

In [20]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67217 entries, P01_01_0 to P37_103_9
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   participant_id       67217 non-null  object
 1   video_id             67217 non-null  object
 2   narration_timestamp  67100 non-null  object
 3   start_timestamp      67217 non-null  object
 4   stop_timestamp       67217 non-null  object
 5   start_frame          67217 non-null  int64 
 6   stop_frame           67217 non-null  int64 
 7   narration            67217 non-null  object
 8   verb                 67217 non-null  object
 9   verb_class           67217 non-null  int64 
 10  noun                 67217 non-null  object
 11  noun_class           67217 non-null  int64 
 12  all_nouns            67217 non-null  object
 13  all_noun_classes     67217 non-null  object
dtypes: int64(4), object(10)
memory usage: 7.7+ MB


In [21]:
df_validation.head()

,participant_id,video_id,narration_timestamp,start_timestamp,stop_timestamp,start_frame,stop_frame,narration,verb,verb_class,noun,noun_class,all_nouns,all_noun_classes
narration_id,,,,,,,,,,,,,,
P01_11_0,P01,P01_11,00:00:00.560,00:00:00.00,00:00:01.89,1,113,take plate,take,0,plate,2,[plate],[2]
P01_11_1,P01,P01_11,00:00:01.700,00:00:01.56,00:00:02.45,93,147,put down plate,put-down,1,plate,2,[plate],[2]
P01_11_10,P01,P01_11,00:00:48.500,00:00:49.15,00:00:50.95,2949,3057,take paper,take,0,paper,49,[paper],[49]
P01_11_100,P01,P01_11,00:05:27.840,00:05:27.28,00:05:31.97,19636,19918,wash cloth,wash,2,cloth,17,[cloth],[17]
P01_11_101,P01,P01_11,00:05:26.840,00:05:27.37,00:05:29.86,19642,19791,take cloth,take,0,cloth,17,[cloth],[17]


In [22]:
df_train100 = df_train[df_train.video_id.apply(lambda x: len(x) == 7)]
df_validation100 = df_validation[df_validation.video_id.apply(lambda x: len(x) == 7)]
print(df_train100.shape) 
print(df_validation100.shape)

(37455, 14)
(0, 14)


In [23]:
df_train100.head()

,participant_id,video_id,narration_timestamp,start_timestamp,stop_timestamp,start_frame,stop_frame,narration,verb,verb_class,noun,noun_class,all_nouns,all_noun_classes
narration_id,,,,,,,,,,,,,,
P01_102_0,P01,P01_102,00:00:01.100,00:00:00.54,00:00:02.23,27,111,take knife and plate,take,0,knife,4,"[knife, plate]","[4, 2]"
P01_102_1,P01,P01_102,00:00:07.600,00:00:07.49,00:00:09.05,374,452,move spatula and colander,move,11,spatula,20,"[spatula, colander]","[20, 39]"
P01_102_10,P01,P01_102,00:00:29.851,00:00:29.66,00:00:30.84,1483,1542,take knife,take,0,knife,4,[knife],[4]
P01_102_100,P01,P01_102,00:03:44.600,00:03:44.45,00:03:45.00,11222,11250,wipe surface,wipe,2,surface,42,[surface],[42]
P01_102_101,P01,P01_102,00:03:45.600,00:03:45.61,00:03:46.01,11280,11300,move glass,move,11,glass,10,[glass],[10]


In [33]:
# Generate new training and validation files from df_train100
# This is to for the pilot study of our model - we will re-evaluate on the entire EPIC-Kitchens dataset later

from sklearn.model_selection import train_test_split

participants = df_train100.participant_id.unique()
p_train, p_test = train_test_split(participants, test_size=0.15)

train = df_train100[df_train100['participant_id'].isin(p_train)]
test = df_train100[df_train100['participant_id'].isin(p_test)]
# train, test = train_test_split(df_train100, test_size=0.15)
# train, val = train_test_split(train, test_size=0.14) # We end up with (train, val, test) = (0.745, 0.105, 0.15) ~ (0.75, 0.10, 0.15)

print('Train participants: ', p_train, '\n')
print('Test participants: ', p_test)

train.info(), test.info()


Train participants:  ['P25' 'P09' 'P03' 'P04' 'P22' 'P01' 'P37' 'P28' 'P35' 'P12' 'P02' 'P23'
 'P07' 'P06' 'P27'] 

Test participants:  ['P26' 'P30' 'P11']


In [7]:
df_train100.reset_index(inplace=True)

In [8]:
df_train100[:25]

,narration_id,participant_id,video_id,narration_timestamp,start_timestamp,stop_timestamp,start_frame,stop_frame,narration,verb,verb_class,noun,noun_class,all_nouns,all_noun_classes
0,P01_102_0,P01,P01_102,00:00:01.100,00:00:00.54,00:00:02.23,27,111,take knife and plate,take,0,knife,4,"[knife, plate]","[4, 2]"
1,P01_102_1,P01,P01_102,00:00:07.600,00:00:07.49,00:00:09.05,374,452,move spatula and colander,move,11,spatula,20,"[spatula, colander]","[20, 39]"
2,P01_102_10,P01,P01_102,00:00:29.851,00:00:29.66,00:00:30.84,1483,1542,take knife,take,0,knife,4,[knife],[4]
3,P01_102_100,P01,P01_102,00:03:44.600,00:03:44.45,00:03:45.00,11222,11250,wipe surface,wipe,2,surface,42,[surface],[42]
4,P01_102_101,P01,P01_102,00:03:45.600,00:03:45.61,00:03:46.01,11280,11300,move glass,move,11,glass,10,[glass],[10]
5,P01_102_102,P01,P01_102,00:03:46.851,00:03:46.06,00:03:46.56,11303,11328,wipe surface,wipe,2,surface,42,[surface],[42]
6,P01_102_103,P01,P01_102,00:03:48.600,00:03:46.71,00:03:47.76,11335,11388,wipe surface,wipe,2,surface,42,[surface],[42]
7,P01_102_104,P01,P01_102,00:03:50.100,00:03:48.21,00:03:49.21,11410,11460,wipe surface,wipe,2,surface,42,[surface],[42]
8,P01_102_105,P01,P01_102,00:03:51.101,00:03:49.91,00:03:54.30,11495,11715,wipe surface,wipe,2,surface,42,[surface],[42]
9,P01_102_106,P01,P01_102,00:03:55.350,00:03:55.22,00:03:55.74,11761,11787,move lid,move,11,lid,6,[lid],[6]


In [9]:
df_validation100.head()

,participant_id,video_id,narration_timestamp,start_timestamp,stop_timestamp,start_frame,stop_frame,narration,verb,verb_class,noun,noun_class,all_nouns,all_noun_classes
narration_id,,,,,,,,,,,,,,


In [10]:
video_info = pd.read_csv('../epic-kitchens-100-annotations/EPIC_100_video_info.csv')
video_info.head()

,video_id,duration,fps,resolution
0,P01_01,1652.152817,59.94006,1920x1080
1,P01_02,502.134967,59.94006,1920x1080
2,P01_03,118.852067,59.94006,1920x1080
3,P01_04,105.238467,59.94006,1920x1080
4,P01_05,1271.988033,59.94006,1920x1080


### Sentence Embeddings

In [12]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('stsb-distilroberta-base-v2')

In [21]:
sentence = ['take knife and plate']
sentence_embedding = sbert_model.encode(sentence, normalize_embeddings=True)
print(f'BERT embedding vector length = {sentence_embedding.shape}')
print(type(sentence_embedding))
print(f'Sentence embedding: {sentence_embedding}')

BERT embedding vector length = (1, 768)
<class 'numpy.ndarray'>
Sentence embedding: [[ 1.47427674e-02 -3.11026089e-02 -6.92465603e-02  7.33942837e-02
  -6.06507296e-03 -2.43162084e-02 -3.54990619e-03 -6.12526461e-02
   1.95246865e-03  1.60941705e-02  3.52159608e-03 -7.26333335e-02
   2.93872897e-02  2.39843894e-02  3.60596143e-02  8.55282396e-02
   5.60207553e-02 -4.49414439e-02 -6.70904852e-03 -7.21871108e-02
   1.15105435e-02  4.95724240e-03  2.70861574e-02 -1.30327931e-02
  -6.00905623e-03  1.96854211e-02 -2.91230083e-02  8.21921416e-03
   9.55284573e-03 -9.51795205e-02  1.41131012e-02 -1.49100199e-02
  -3.57156135e-02  4.15555350e-02  5.02955206e-02  3.96701135e-02
  -1.36269573e-02  2.05318332e-02 -4.50411439e-02 -2.16201302e-02
  -3.81516628e-02  7.89472312e-02 -1.79788657e-02 -3.49641889e-02
   3.64372618e-02 -2.03096401e-02 -1.48022926e-04 -1.76182799e-02
   4.80667409e-03  5.83866723e-02 -4.78191562e-02  5.26288003e-02
   1.79953687e-02 -1.96166672e-02  2.41080429e-02  4.59337

### Extract training frames

In [10]:
def write_pickle(o, pname):
    with open(pname, 'xb') as handle:
        pickle.dump(o, handle)

def read_pickle(pname, single=True):
    with open(pname, 'rb') as handle:
        if single:
            return pickle.load(handle)
        data = []
        while True:
            try:
                data.append(pickle.load(handle))
            except EOFError:
                return data


In [4]:
# Testing code
import torch
import pickle
data = read_pickle('../clip_rgb_features.pickle')
# print(df)
for idx, vals in data.items():
    print(f'{idx}: {vals.shape}')

0: torch.Size([325, 512])
1: torch.Size([1112, 512])
2: torch.Size([9738, 512])
3: torch.Size([50, 512])
4: torch.Size([62, 512])
5: torch.Size([88, 512])
6: torch.Size([75, 512])
7: torch.Size([50, 512])
8: torch.Size([212, 512])
9: torch.Size([1980, 512])


In [11]:
def get_sec(time_str):
    """Get Seconds from time."""
    h, m, s = time_str.split(':')
    return float(h) * 3600 + float(m) * 60 + float(s)

In [12]:
def extract_CLIP_features(video_path):
    feature_type = 'clip'
    model_name = 'ViT-B/32'

    args = OmegaConf.load(build_cfg_path(feature_type))
    args.feature_type = feature_type
    # args.video_paths = video_path
    args.device_type = 'cuda:1'
    args.batch_size = 32

    extractor = ExtractCLIP(args)
    feats = extractor.extract(video_path)
    return feats

In [ ]:
df = df_train100.copy()
df['video_path'] = '../2g1n6qdydwa9u22shpxqzp0t8m/' + df.participant_id + '/videos/' + df.video_id + '.MP4'
feat_dict = {}
prev = 0
for index, row in df.iterrows():
    # print(index)
    # index = int(index)
#     if index == 2:
#         print(f'Exiting')
#         break
    video_id, video_path, narr_timestamp = row['video_id'], row['video_path'], row['narration_timestamp']
    duration = video_info.loc[video_info['video_id'] == video_id]['duration'].iat[0]
    start = get_sec(narr_timestamp)
    start = max(start - 2, 0)
    end = get_sec(df.at[index + 1, 'narration_timestamp']) if index < len(df) - 1 else duration
    command = f'ffmpeg -ss {start} -i {video_path} -c copy -to {end} temp.mp4 -y'
    subprocess.call(command, shell=True)
    # ffmpeg_extract_subclip(video_path, start, end, targetname='temp.MP4')
    feat_dict[index] = extract_CLIP_features('temp.mp4')
    os.remove('temp.mp4')
    if (index % 1000 == 0 and index > 0) or (index == len(df) - 1):
        out_file = '../pickles/' + str(prev + 1) + '_to_' + str(index) + '.pkl' # name = {previous_index}_to_{current_index}.pkl
        print(f'Dumping to pickle from index = {prev + 1} to index = {index}')
        with open(out_file, 'xb') as handle:
            pickle.dump(feat_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
        prev = index
        feat_dict = {}
        print(f'Created pickle file at {out_file}')

ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.5.0 (crosstool-NG 1.24.0.131_87df0e6_dirty)
  configuration: --prefix=/coc/pcba1/bdas31/Miniconda3/envs/torch-zoo --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1596712246804/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
  libpostproc    55.  7.100 / 55.  7.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 

frame=11273 fps=222 q=-1.0 Lsize=  665912kB time=00:03:44.58 bitrate=24289.5kbits/s speed=4.43x    
video:660512kB audio:5212kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.028107%
ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.5.0 (crosstool-NG 1.24.0.131_87df0e6_dirty)
  configuration: --prefix=/coc/pcba1/bdas31/Miniconda3/envs/torch-zoo --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1596712246804/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '../2g1n6qdydwa9u22shpxqzp0t8m/P01/videos/P01_102.MP4':
  Metadata:
    major_brand     : mp41
    minor_version   : 538120216
    compatible_brands: mp41
    creation_time   : 2019-05-20T20:57:29.000000Z
    firmware        : HD7.01.01.70.00
  Duration: 00:04:34.92, start: 0.000000, bitrate: 24253 kb/s
    Stream #0:0(eng): Video: hevc (Main) (hvc1 / 0x31637668), yuvj420p(pc, bt709), 1920x1080 [SAR 1:1 DAR 16:9], 24001 kb/s, 50 fps, 50 tbr, 90k tbn, 50 tbc (default)
    Metadata:
      creation_time   : 2019-05-20T20:57:29.000000Z
      handler_name    : GoPro H.265
      encoder         : GoPro H.265 encoder
      timecode        : 20:57:29:07
    Stream #0:1(eng): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 189 kb/s (default)
    Metadata:
      creation_time   : 2019-05-20T20:57:29.000000Z
      handler_name    : GoPro AAC  
      timecode        : 20:57:29:07
    Stream #0:2(eng): Data: none (tmcd / 0x64636D74) (default)
   

frame= 2346 fps=920 q=-1.0 Lsize=  138593kB time=00:00:46.80 bitrate=24259.0kbits/s speed=18.3x    
video:137467kB audio:1085kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.029169%
ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.5.0 (crosstool-NG 1.24.0.131_87df0e6_dirty)
  configuration: --prefix=/coc/pcba1/bdas31/Miniconda3/envs/torch-zoo --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1596712246804/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100

frame= 1996 fps=463 q=-1.0 Lsize=  117725kB time=00:00:39.55 bitrate=24383.6kbits/s speed=9.17x    
video:116768kB audio:923kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.029451%
ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.5.0 (crosstool-NG 1.24.0.131_87df0e6_dirty)
  configuration: --prefix=/coc/pcba1/bdas31/Miniconda3/envs/torch-zoo --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1596712246804/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100


...

Detect missing frame


ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.5.0 (crosstool-NG 1.24.0.131_87df0e6_dirty)
  configuration: --prefix=/coc/pcba1/bdas31/Miniconda3/envs/torch-zoo --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1596712246804/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
  libpostproc    55.  7.100 / 55.  7.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 

Output #0, mp4, to 'temp.mp4':
  Metadata:
    major_brand     : mp41
    minor_version   : 538120216
    compatible_brands: mp41
    firmware        : HD7.01.01.70.00
    encoder         : Lavf58.45.100
    Stream #0:0(eng): Video: hevc (Main) (hvc1 / 0x31637668), yuvj420p(pc, bt709), 1920x1080 [SAR 1:1 DAR 16:9], q=2-31, 24002 kb/s, 0.02 fps, 50 tbr, 90k tbn, 90k tbc (default)
    Metadata:
      creation_time   : 2019-05-21T09:00:45.000000Z
      handler_name    : GoPro H.265
      encoder         : GoPro H.265 encoder
      timecode        : 09:00:45:37
    Stream #0:1(eng): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 189 kb/s (default)
    Metadata:
      creation_time   : 2019-05-21T09:00:45.000000Z
      handler_name    : GoPro AAC  
      timecode        : 09:00:45:37
Stream mapping:
  Stream #0:0 -> #0:0 (copy)
  Stream #0:1 -> #0:1 (copy)
Press [q] to stop, [?] for help
frame=  948 fps=602 q=-1.0 Lsize=   55942kB time=00:00:18.09 bitrate=25333.0kbits/s speed=

ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.5.0 (crosstool-NG 1.24.0.131_87df0e6_dirty)
  configuration: --prefix=/coc/pcba1/bdas31/Miniconda3/envs/torch-zoo --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1596712246804/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
  libpostproc    55.  7.100 / 55.  7.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 

frame= 4232 fps=1849 q=-1.0 Lsize=  250009kB time=00:01:24.28 bitrate=24300.3kbits/s speed=36.8x    
video:247981kB audio:1957kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.028568%
ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.5.0 (crosstool-NG 1.24.0.131_87df0e6_dirty)
  configuration: --prefix=/coc/pcba1/bdas31/Miniconda3/envs/torch-zoo --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1596712246804/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.10

In [ ]:
print(len(feat_dict))

In [ ]:
for f in feat_dict:
    x = feat_dict[f]['clip']
    print(x.shape)

In [59]:
data = []
with open(output_path, 'rb') as f:
    while True:
        try:
            data.append(pickle.load(f))
        except EOFError:
            break

print(data)

[{1: 'a'}, {2: 'a'}]


## Testing out model architecture

Assume at this point, we have the fused frame features

In [ ]:
import torch
from torch import nn as nn
from torch.nn import functional as F

In [ ]:
# Pass through conv1d layer
F = nn.Conv1d(f, kernel_size=3, padding='same') # check args
verb_dict = clip.model_encode(verb_list) # low-key pseudo code

W = verb_dict.tensor() # C_verb x d
A = F.sigmoid(W.dot(F))
A = F.normalize(A, dim=1)

A_i = A[c, :]

Fi = A*Fi / torch.sum(A)